### Title: "MAL 03"

| Gruppe 30 |
| --------- |

| Navn                    | Studienummer |
| ----------------------- | ------------ |
| Lasse Borring Petersen  | 202208165    |
| Benjamin Harboe Strunge | 202209864    |
| Esben Inglev            | 202210050    |
| Asbjørn Vad             | 202208512    |


# CNN


# Learning Curves



# Grid Search



# NanoGPT

## Qa, Qb (and some Qc)

### Introduction
In this exercise, we take a step further into the capabilities of neural networks, with the addition of GPT's. I the exercise, we train a model that can guess the next character to be written, with training from a specific body of text. 
The GPT works by looking at a window of inputs, of a certain size, and the produces output is a probability distribution of the next most likely token. In larger LLM's, these tokens are whole or parts of words, but in our case, they are just single characters.

### HCA training
The process of training the HCA data, with the AU GPU cluster, starts by first preparing the data, with `make prepare`.
```makefile
.PHONY:prepare
prepare:
	...
	@ test -f data/$(DATASET)/train.bin  || (echo "RUNNING  data/$(DATASET)/prepare.py.." && cd data/$(DATASET) && $(PYTHON) ./prepare.py)
	@ echo PREPARE: all ok!
```

This last step of the prepare command calls the prepare.py file, which basically just creates mappings of characters into interger id's. This is the tokenizing. 

Then comes the training part, with `make train`:
```
.PHONY:train
train:
	@# /home/shared/gpt_node_ok.sh
	$(PYTHON) train.py config/train_$(DATASET).py --max_iters=10 --warmup_iters=1 --eval_iters=1 --eval_interval=1
	@ $(MAKE) -s precondition_predict
	@# more parameters --device=mps --compile=False --eval_iters=20 --log_interval=1 --block_size=64 --batch_size=12 --n_layer=4

```
This command initiates the training, and hold a bunch of training parameters that we know from earlier NN trainings. Here's a brief explanation of some of them:
- `max_iters`: This is the maximum number of training iterations/epochs that the model goes through, before training is stopped.
- `block_size`: This is the size of the window of inputs, also called the context, that the model can have.
- `batch_size`: The batch size is the number of training 'examples' from the data, that the model is trained on each iteration.
- `gradient_accumulation_steps`: This is the number of training iterations the model will go through, while accumulating the gradients. After this amount of iterations, only then are the weights updated. This effectively makes the batch_size bigger, and is nice when memory is limited. 

These configurations all play together in the training. Each iteration, the model is given `batch_size` amount of training examples. For each of the examples, the model tries to predict the next character, for each sequence in the `block_size`.
- Based on char 1, it tries to predict char 2
- Based on char 1-2, it tries to preduct char 3
- ...
So a total of `batch_size` * `block_size` amount of guesses.

Inside the `config/train_hca.py` file, there's more configuration parameters, some of which specify the transformer setup. Here's a few of them:
- `n_embd`: This is the size of the embeddings of the tokens. An embedding is a vector, which represents the token. So 192, means that each token is represented by a 192 dimension vector.
- `n_layer`: This is the amount of layers in the transformer. Each layer processes the input sequentially, before the final prediction is made from the last layer.
- `n_head`: This is the amount of attention heads. All head are used in each layer, and learns to attend to different patterns. Some heads might attend to the nearby characters, and other might look further back or attend to the whole sentence.
- `dropout`: This is the same as in normal neural networks, where a percentage of the nodes are turned off each iteration, in order to avoid some nodes gaining too much significance.
- `learning_rate`: Learning rate is well known from our other ML studies, and specifies how aggressively we adjust the weights after each iteration, during the backwards pass.

During training, we see continuous loss outputs, and sometimes also validation loss outputs. Lower values mean, that the model gives high probability to the right answer, so that is what we want. Validation loss is calculated at some points during training, and shows how the model performes on unseen data.

### HCA Predicting
When the model is trained, it's time to predict. 
```makefile
.PHONY:pred
pred: precondition_predict
	$(PYTHON) sample.py --out_dir=out-$(DATASET) --temperature=0.8 --num_samples=2 --start="Der kom en solda"
```

This make command initiates the prediction. We see the start prompt, "Det kom en solda", and the model will try and predict the next characters.

The temperature is a scaler for the predictions. Lower values will make the model be more likely to choose among it's top most predicted tokens, while larger values will make the model choose more erratic or seemingly random tokens.

Inside the sample.py file, there's also options for setting the amount/length of the output, ie, how many new tokens should be generated.

### Other LLM Lingo

Here's an explanation of some more ML lingo when talking about GPT's and LLM's:
- Design matrix $\mathbf{X}$: This is the input to the model, so basically whatever is in the input.txt file.
- Test data $\mathbf{y}_\textrm{test}\ $: The test data exists inherintly in the input, because the answer to every series of tokens, is the next token in the input. In this sense, the data is auto-labeled.
- Prompting with LLM's: Prompting is the input we feed to the LLM, or more preciesly, the art of shaping the input in order for the LLM to work as we want it and make it generate the desired output. 

### Our own data

For training on our own data, we choose the Pokemon Wikipedia page. https://en.wikipedia.org/wiki/Pok%C3%A9mon

The text is put into a input.txt file, and we followed the setup guide from the `make help` command. During training, we set the model to 1000 iterations, and tested the result. Here's an output from the training:

![Training Start](img/nanotraining1.png)
![Training End](img/nanotraining2.png)

We see here that we end up with a loss of around 1.449.

Here's a prompt with a temperature of 1.0:

![Temp 1.0](img/predict1.png)

Below is the result of a temperature of 0.1:

![Temp 0.1](img/predict2.png)

Even though the output is not coherent with lower temperature, it is easy to see the different in character/token choice the model makes. It's actual words comming out of the second attempt!

### Conclusion

Even though we didn't expect the model to generate any great output, it's still impressive to see how well it performs with such minimal training. It's only trained on a few pages of input, the training lasts under a minute, and still it's able to produce output that is spelling words correctly, and also gives some hints to the theme of the input text.
No wonder that the the whole of Reddit, Wikipedia, Stack overflow, news articles etc, fed into massive datasets thats trained for days or weeks, can produce the LLM's that seem so clever!